In [1]:
import numpy as np
import movielensutils

%load_ext autoreload
%autoreload 2

# Question 1.1
Récupérer la base de données Movielens [HKBR99] sur le site suivant :

http://files.grouplens.org/datasets/movielens/ml-100k.zip.

Lancer la fonction load movielens de movielens utils.py avec le bon nom de fichier et

vérifier que la matrice R retournée est bien de taille 943×1 682. Que fait l’option minidata ?

In [2]:
%%time
R, mask = movielensutils.load_movielens('./ml-100k/u.data', 
                                        minidata=False)

CPU times: user 828 ms, sys: 12 ms, total: 840 ms
Wall time: 845 ms


In [3]:
R.shape

(943, 1682)

## Reponse 1.1

L'option minidata, si True le fonction va retourner le data tronqués avec 100 utilisateurs et 200 films R[0:100, 0:200]


# Question 1.2
Combien y a-t-il d’utilisateurs, de films référencés dans la base de données ? Quel est le
nombre total de notes ?

In [4]:
print("nombre d'utilisateurs: %d" % R.shape[0])
print("nombre de films: %d" % R.shape[1])
print("nombre total de notes %d" % np.sum(R > 0))

nombre d'utilisateurs: 943
nombre de films: 1682
nombre total de notes 100000


# Question 1.3
La fonction objectif est-elle convexe ?  

$$ 
L(P,Q) =  \frac{1}{2}\left|\left|\mathbb{1}_K \circ (R - QP) \right|\right|_F^2 + \frac{\rho}{2}||Q||_F^2 + \frac{\rho}{2}||P||_F^2
$$

**La fonction est deux fois dérivable, elle est convexe ssi Hessian est défini positive. Ob va vérifier le gradient pour le cas ou chaque variable est scalaire**

$$
l(p, q) = \frac{1}{2} (r - qp)^2 + \frac{\rho}{2}q^2 + \frac{\rho}{2}p^2 \\
$$

On va calculer Hessian de $\frac{1}{2} (r - qp)^2$ parce que l'autre partie est convexe.

$$
\begin{pmatrix}
                    q^2 && qp - r \\ 
                    qp-r &&p^2
\end{pmatrix}
$$

Ce forme quadratique n'est pas défini positive. **$L$ - n'est pas convexe**


Quel est son gradient ? 

On calcule le gradient en utilisant le proptiétés de matrice [Matrix Cookbook](http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/3274/pdf/imm3274.pdf)

$$ 
\frac{\partial L}{\partial P} = - Q^T\left[\mathbb{1}_K\circ (R - QP)\right] + \rho P \\
\frac{\partial L}{\partial Q} = - \left[\mathbb{1}_K\circ (R - QP)\right] P^T + \rho Q
$$


Est-il lipschitzien ? 

** Non, parce que l'accroissement du gradiant est plus grand que la fonction lineaire d'accroissement des arguments **

Donner la constante de Lipschitz le cas échéant. 
On pourra remarquer que la fonction que l’on cherche à minimiser est un polynôme.

# Question 2.1
La fonction objectif g est-elle convexe ? 

$$
g(P) = \frac{1}{2} ||1_K \circ (R - Q^0P)||^2_F + \frac{\rho}{2} ||Q^0||^2_F + \frac{\rho}{2} ||P||^2_F \\
P^1 = arg \min_P g(P)
$$

g est convexe parce que le Hessian est défini positive qvec propre $\rho$

$$\bigtriangledown^2 g(P) = {Q^0}^T \mathbb{1}_K \mathbb{1}_K^T {Q^0} + \rho I$$ 

Quel est son gradient ? 

$$ \frac{\partial L}{\partial P} = - (Q^0)^T\left[\mathbb{1}_K\circ (R - Q^0P)\right] + \rho P $$

On admettra que le gradient
est lipschitzien de constante $L_0 = \rho + ||(Q^0)^T Q^0||_F .$

***

# Trouver P quand $Q_0$ fixé
On initialise l’algorithme de résolution avec $Q_0$ (resp. $P_0$) défini comme les F vecteurs
singuliers à gauche (resp. à droite) de R associés à ses $|C|$ plus grandes valeurs singulières.
Les valeurs manquantes de R seront fixées à 0 pour cette étape d’initialisation. Vous pourrez
utiliser la fonction **scipy.sparse.linalg.svds**.

In [5]:
import scipy
import scipy.sparse.linalg as ssl

In [6]:
C = 4
rho = 0.3

In [7]:
Q0, sigmas, P0 = ssl.svds(A=R, k=C)

# Question 2.2
La fonction fournie *objective* calcule g(P ). 

Compléter cette fonction pour qu’elle calcule aussi ∇g(P ). 

Vous pourrez vérifier votre calcul avec la fonction **scipy.optimize.check grad**
(vous aurez peut-être besoin des fonctions **numpy.reshape** et **numpy.ravel** car check grad ne gère pas les variables matricielles).

Check grad

In [8]:
def func(P, Q0, R, mask, rho):
    tmp = (R - Q0.dot(P)) * mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    return val

def gradfunc(P, Q0, R, mask, rho):
    tmp = (R - Q0.dot(P)) * mask
    grad_P = - Q0.T.dot(tmp) + rho * P
    return grad_P

In [9]:
f = lambda P: np.ravel(func(P=P.reshape(C, -1), 
                         Q0=Q0, 
                         R=R, 
                         mask=mask, rho=rho))

grad = lambda P: np.ravel(gradfunc(P=P.reshape(C, -1), 
                                Q0=Q0, 
                                R=R, 
                                mask=mask, rho=rho))

In [10]:
%%time
error = scipy.optimize.check_grad(func=f, grad=grad, x0=np.ravel(P0))
relative_error = error / np.linalg.norm(grad(P0))
print("grad error = %f" % error)
print("rel grad error = %f" % relative_error)

grad error = 1.120187
rel grad error = 0.001523
CPU times: user 6min 16s, sys: 12min 37s, total: 18min 54s
Wall time: 2min 47s


# Question 2.3
Coder une fonction **gradient(g, P0, gamma, epsilon)** qui minimise une fonction g par la
méthode du gradient à pas constant $\gamma$ en partant du point initial $P_0$ et avec le critère d’arrêt
$$ || \nabla g(P_k)||_F ≤ \epsilon. $$

In [11]:
def gradient(g, P0, gamma, epsilon, verbose=False):
    """
    La fonction du minimization de g 
    avec methode du descente du gradient 
    g : fonction qui retourne obj, grad 
    P0 : la variable matricielle de taille C x I
    gamma : taux d'apprentissage
    epsilon : condition d'arret
    
    Sorties :
    P : la matrice de taille C x I qui minimize g
    """
    P = P0
    k = 0
    while True:
        obj, grad = g(P)
        if np.linalg.norm(grad) < epsilon:
            break
        P = P - gamma * grad
        k += 1
        if verbose:
            print("Iter: %d, gamma: %f" % (k, gamma))
            
    
    return P

# Question 2.4
Utiliser la fonction codée à la question précédente pour minimiser la fonction g jusqu’à la
précision $\epsilon = 1.$

In [12]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = - Q0.T.dot(tmp) + rho * P 

    return val, grad_P

In [13]:
g = lambda P: objective(P=P, 
                        Q0=Q0, 
                        R=R, 
                        mask=mask, rho=rho)
L = rho + np.linalg.norm(Q0.T.dot(Q0), ord='fro')
epsilon = 1
gamma = 1./L

In [14]:
%%time
P = gradient(g=g, P0=P0, gamma=gamma, epsilon=epsilon, verbose=True)

Iter: 1, gamma: 0.434783
Iter: 2, gamma: 0.434783
Iter: 3, gamma: 0.434783
Iter: 4, gamma: 0.434783
Iter: 5, gamma: 0.434783
Iter: 6, gamma: 0.434783
Iter: 7, gamma: 0.434783
Iter: 8, gamma: 0.434783
Iter: 9, gamma: 0.434783
Iter: 10, gamma: 0.434783
Iter: 11, gamma: 0.434783
Iter: 12, gamma: 0.434783
Iter: 13, gamma: 0.434783
Iter: 14, gamma: 0.434783
Iter: 15, gamma: 0.434783
Iter: 16, gamma: 0.434783
Iter: 17, gamma: 0.434783
Iter: 18, gamma: 0.434783
Iter: 19, gamma: 0.434783
Iter: 20, gamma: 0.434783
Iter: 21, gamma: 0.434783
Iter: 22, gamma: 0.434783
Iter: 23, gamma: 0.434783
Iter: 24, gamma: 0.434783
Iter: 25, gamma: 0.434783
Iter: 26, gamma: 0.434783
CPU times: user 1.41 s, sys: 2.27 s, total: 3.68 s
Wall time: 626 ms


In [15]:
g(P)

(369551.54991481942,
 array([[ 0.00085937,  0.00129404,  0.00969936, ...,  0.00341002,
         -0.0002994 , -0.00612912],
        [-0.00080519, -0.00702478, -0.0072371 , ..., -0.00261146,
          0.00240894,  0.00125411],
        [ 0.00050192, -0.00033308,  0.00320318, ...,  0.00246802,
         -0.00062718, -0.00109561],
        [-0.0002689 , -0.00561367, -0.01044149, ..., -0.0004381 ,
         -0.00516698, -0.00447528]]))

***

# Raffinements algorithmiques pour le problème à $Q_0$ fixé

# Question 3.1
Rajouter une méthode de recherche linéaire à la méthode du gradient, de manière à s’affranchir du calcul de la constante de Lipschitz.

In [16]:
def linesearch(gamma, f, gradf, x0, a=0.5, b=None):
    """
    linesearch method to find learning_rate (gamma)

    gamma = argmin f( xk - gamma * df(xk))
    """
    if b is None:
        b = 2 * gamma

    assert 0 < a < 1
    assert b > 0

    l = 0
    x = x0

    def xplus(gamma, x, gradf):
        return x - gamma * gradf(x)

    fx = f(x)
    gradfx = gradf(x)
    
    while True:
        gamma = b * np.power(a, l)
        xp = xplus(gamma, x, gradf)
        lhs = f(xp)
        
        rhs = fx + np.vdot(gradfx, xp - x) + \
            np.linalg.norm(x - xp)**2 / (2 * gamma)
    
        if lhs <= rhs:
            break

        l += 1

    return b * np.power(a, l)

In [17]:
def gradient_with_linesearch(g, P0, gamma, epsilon, 
                             a=0.5, b=0.5, verbose=False):
    """
    La fonction du minimization de g 
    avec methode du descente du gradient 
    g : fonction qui retourne obj, grad 
    P0 : la variable matricielle de taille C x I
    gamma : taux d'apprentissage
    epsilon : condition d'arret
    a, b : parameteres de recherche lineaire
    
    Sorties :
    P : la matrice de taille C x I qui minimize g
    """
    P = P0
    k = 0
    while True:
        gamma = linesearch(gamma, 
                            lambda P: g(P)[0], 
                            lambda P: g(P)[1], 
                            P, a, b)
        
        obj, grad = g(P)
        if np.linalg.norm(grad) < epsilon:
            break
        P = P - gamma * grad
        
        k += 1
        if verbose:
            print("Iter: %d, gamma: %f, ||grad|| = %f" % (k, gamma, 
                                                          np.linalg.norm(grad)))
            print("Obj: %f" % obj)
        
    return P

In [18]:
%%time
Pls = gradient_with_linesearch(g=g, 
                               P0=P0, 
                               gamma=gamma, 
                               epsilon=epsilon, verbose=True)

Iter: 1, gamma: 0.500000, ||grad|| = 735.422318
Obj: 685091.782648
Iter: 2, gamma: 0.500000, ||grad|| = 406.726326
Obj: 476377.423776
Iter: 3, gamma: 0.500000, ||grad|| = 240.622838
Obj: 410948.625792
Iter: 4, gamma: 0.500000, ||grad|| = 151.250253
Obj: 387501.076358
Iter: 5, gamma: 0.500000, ||grad|| = 99.963810
Obj: 378043.960249
Iter: 6, gamma: 0.500000, ||grad|| = 68.729358
Obj: 373842.953513
Iter: 7, gamma: 0.500000, ||grad|| = 48.718652
Obj: 371830.491180
Iter: 8, gamma: 0.500000, ||grad|| = 35.360362
Obj: 370808.672326
Iter: 9, gamma: 0.500000, ||grad|| = 26.145940
Obj: 370265.902462
Iter: 10, gamma: 0.500000, ||grad|| = 19.621897
Obj: 369967.168039
Iter: 11, gamma: 0.500000, ||grad|| = 14.904837
Obj: 369797.997722
Iter: 12, gamma: 0.500000, ||grad|| = 11.435576
Obj: 369699.944954
Iter: 13, gamma: 0.500000, ||grad|| = 8.847867
Obj: 369642.005529
Iter: 14, gamma: 0.500000, ||grad|| = 6.894868
Obj: 369607.208324
Iter: 15, gamma: 0.500000, ||grad|| = 5.406160
Obj: 369586.018171
Ite

In [19]:
g(Pls)

(369551.66671745956,
 array([[ 0.00082624,  0.00137667,  0.0103768 , ...,  0.00361737,
         -0.00031751, -0.00650227],
        [-0.00076899, -0.00742393, -0.00770228, ..., -0.00277026,
          0.00255467,  0.00133046],
        [ 0.00048618, -0.00029021,  0.00332605, ...,  0.00261809,
         -0.00066512, -0.00116231],
        [-0.00025361, -0.00588765, -0.01096729, ..., -0.00046474,
         -0.00547956, -0.00474774]]))

# Question 3.2
Justifiez que l’on peut utiliser la méthode du gradient conjugué pour ce problème et codez-la.

## Reponse 
Pour utiliser la méthode du gradient conjugé on doit vérifier que la marice $A$ de la forme quadratique est symétrique est défini positive.

On developpe la fonction de perte en utilisant [les propriétes de matrice transposé](https://math.nyu.edu/~neylon/linalgfall04/project1/dj/proptranspose.htm)

$ g(P) = \frac{1}{2}\left|\left|
\mathbb{1}_K \circ (R - Q^0P) \right|\right|_F^2 + \frac{\rho}{2}\left|\left|
Q^0\right|\right|_F^2 + \frac{\rho}{2}\|
P \|_F^2 = \frac{1}{2} \left( \left(\mathbb{1}_K \circ (R - Q^0P)\right)^T\mathbb{1}_K \circ(R - Q^0P)\right) + \frac{\rho}{2}\left( {Q^0}^TQ^0 \right) + \frac{\rho}{2}\left( P^TP \right) = \frac{1}{2} P^T \left(  ( {Q^0}^T  (\mathbb{1}_K \mathbb{1}_K^T) {Q^0}) + \rho I\right) P - \frac{1}{2} \left( P^T{Q^0}^T(\mathbb{1}_K \circ R) + (\mathbb{1}_K \circ R)^TQP \right) + \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q^0}^TQ^0\right)$

Alor $g(P) = \frac{1}{2}P^TAP + 0.5*b^TP + 0.5*P^Tb + c$ avec 
$$\begin{align*} 
A &= ({Q^0}^T   (\mathbb{1}_K \mathbb{1}_K^T) {Q^0}) + \rho I \\ 
b &= -(Q^0)^T(\mathbb{1}_K \circ R)
\\ c &= \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q^0}^TQ^0\right)
\end{align*}$$

le gradient $\triangledown g(P) = AP + b$

In [20]:
def calcAd(d, Q, mask, rho):
    tmp = Q.dot(d) * mask
    Ad = rho * d + Q.T.dot(tmp)
    return Ad

In [21]:
def conjugated_gradients(f, calcAd, P0, epsilon, verbose=False):
    """
    la méthode du gradient conjugué adopté au forme matriciel
    

    Sorties:
    P - minimum de la fonction :f:
    """
    gradf = lambda P: f(P)[1]
    
    
    g = gradf(P0)
    d = - g
    Ad = calcAd(d)
    s = - np.sum(d * g) / np.sum(d * Ad)
    
    x = P0 + s * d
    k = 0
    

    while True:
        g = gradf(x)
        
        b = np.sum(Ad * g) / np.sum(d * Ad)
        d = - g + b*d
        
        Ad = calcAd(d)
        
        s = - np.sum(d * g) / np.sum(d * Ad)
        x = x + s * d
        
        if np.linalg.norm(g) < epsilon:
            break
        
        k += 1        
        if verbose:
            print("Iter: %d, gamma: %f" % (k, s))

    return x

In [22]:
%%time
Pgc = conjugated_gradients(f=g, 
                           calcAd=lambda x: calcAd(x, Q0, mask, rho), 
                           P0=P0, 
                           epsilon=1, verbose=True)

Iter: 1, gamma: 1.300122
Iter: 2, gamma: 1.359458
Iter: 3, gamma: 1.376762
Iter: 4, gamma: 1.435303
Iter: 5, gamma: 1.425017
CPU times: user 576 ms, sys: 1.27 s, total: 1.85 s
Wall time: 351 ms


In [23]:
g(Pgc)

(369550.62772981054,
 array([[ -2.60394665e-03,  -2.57856660e-04,   6.52643864e-04, ...,
           8.64899900e-04,  -8.60404716e-05,  -1.49984883e-03],
        [  3.65922819e-03,   2.99785399e-03,   8.84160220e-07, ...,
          -6.62465785e-04,   6.92325411e-04,   3.06950366e-04],
        [ -3.45216860e-03,   2.62011263e-04,  -1.68266281e-03, ...,
           6.26108085e-04,  -1.80253504e-04,  -2.68171834e-04],
        [ -5.46486737e-03,  -4.10463729e-03,   3.33928619e-03, ...,
          -1.11167115e-04,  -1.48515993e-03,  -1.09572642e-03]]))

# Question 3.3
Comparer les performances des trois algorithmes.

## Reponse 


In [24]:
%%time
P = gradient(g=g, P0=P0, gamma=gamma, epsilon=epsilon, verbose=False)

CPU times: user 1.7 s, sys: 3.36 s, total: 5.07 s
Wall time: 854 ms


In [25]:
%%time
Pls = gradient_with_linesearch(g=g, 
                               P0=P0, 
                               gamma=gamma, 
                               epsilon=epsilon, verbose=False)

CPU times: user 9.11 s, sys: 19 s, total: 28.1 s
Wall time: 4.69 s


In [26]:
%%time
Pgc = conjugated_gradients(f=g, 
                           calcAd=lambda x: calcAd(x, Q0, mask, rho), 
                           P0=P0, 
                           epsilon=1, verbose=False)

CPU times: user 768 ms, sys: 1.56 s, total: 2.33 s
Wall time: 413 ms


In [27]:
print("méthode du gradient: %f" % g(P)[0])
print("recherche linéaire: %f" % g(Pls)[0])
print("gradient conjugué: %f" % g(Pgc)[0])

méthode du gradient: 369551.549915
recherche linéaire: 369551.666717
gradient conjugué: 369550.627730


| méthod | temps | # pas |
|---|---|---|
| méthode du gradient  |  579ms | 26  |
| recherche linéaire  | 2.25 s  | 22  |
| gradient conjugué | 239 ms |  5 | 



---

# Résolution du problème complet

## Question 4.1 
Résoudre le problème (1) par la méthode du gradient avec recherche linéaire jusqu’à la
précision $\epsilon$ = 100. À quoi correspond ce qui est retourné par l’algorithme ?

In [28]:
R, mask = movielensutils.load_movielens('./ml-100k/u.data', 
                                        minidata=False)
C = 4
rho = 0.3
Q0, sigmas, P0 = ssl.svds(A=R, k=C)

PQvec = np.concatenate([Q0.ravel(), P0.ravel()])
g = lambda PQvec : movielensutils.total_objective_vectorized(PQvec, 
                                                             R, 
                                                             mask, 
                                                             rho)

In [29]:
%%time
newPQvec = gradient_with_linesearch(g=g, 
                             P0=PQvec, 
                             gamma=gamma, 
                             epsilon=100, verbose=True)

Iter: 1, gamma: 0.125000, ||grad|| = 330.765122
Obj: 686287.214881
Iter: 2, gamma: 0.001953, ||grad|| = 13150.957263
Obj: 401608.166951
Iter: 3, gamma: 0.000488, ||grad|| = 12552.903976
Obj: 213832.379676
Iter: 4, gamma: 0.001953, ||grad|| = 6256.538803
Obj: 163269.118757
Iter: 5, gamma: 0.001953, ||grad|| = 4356.863571
Obj: 106978.855916
Iter: 6, gamma: 0.000488, ||grad|| = 5335.243852
Obj: 87832.659676
Iter: 7, gamma: 0.003906, ||grad|| = 2646.626860
Obj: 78513.670304
Iter: 8, gamma: 0.000488, ||grad|| = 3939.427833
Obj: 64238.663254
Iter: 9, gamma: 0.000977, ||grad|| = 1730.354792
Obj: 59052.447403
Iter: 10, gamma: 0.001953, ||grad|| = 1352.896128
Obj: 56977.231986
Iter: 11, gamma: 0.000977, ||grad|| = 1508.507467
Obj: 54401.478214
Iter: 12, gamma: 0.000977, ||grad|| = 1196.642901
Obj: 53103.191140
Iter: 13, gamma: 0.001953, ||grad|| = 1037.149108
Obj: 52100.350524
Iter: 14, gamma: 0.000488, ||grad|| = 1395.657252
Obj: 50810.693037
Iter: 15, gamma: 0.007812, ||grad|| = 808.323271
Ob

Iter: 123, gamma: 0.000488, ||grad|| = 158.782332
Obj: 35980.241828
CPU times: user 3min 43s, sys: 8min 2s, total: 11min 46s
Wall time: 1min 46s


In [30]:
g(newPQvec)

(35972.554792992269,
 array([ 0.0250741 ,  0.08463934, -0.03628724, ..., -1.62290103,
         4.88764032, -2.04180079]))

In [31]:
def unpack(PQvec, R):
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = int(PQvec.shape[0] / (n_items + n_users))
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))
    return P, Q

In [32]:
newP, newQ = unpack(newPQvec, R)

## Question 4.2
Quand Q (resp. P ) est fixé, le problème est facile à résoudre. La méthode des moindres carrés
alternés utilise ce fait et consiste en l’algorithme suivant :

## Reponse

A P fixé (Q fixé) le problème est convex et l'objectif va décroitre.

# Question 4.3
Coder la méthode des moindres carrés alternés.

In [33]:
def objective_Q(P0, Q, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q.dot(P0)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P0 ** 2))

    grad_Q = - tmp.dot(P0.T) + rho * Q 

    return val, grad_Q

In [34]:
def alternating_gradient(objp, objq, 
                         P0, Q0, gamma, epsilon, 
                         verbose=False, max_iter=30):
    
    Qk = Q0
    Pk = P0
    
    for k in range(max_iter):
                                    
        Pk = gradient_with_linesearch(g=lambda P: gp(P, Qk), 
                                      P0=Pk, 
                                      gamma=gamma, 
                                      epsilon=0.2*epsilon, 
                                      verbose=verbose)
        
        Qk = gradient_with_linesearch(g=lambda Q: gq(Pk, Q), 
                                      P0=Qk, 
                                      gamma=gamma, 
                                      epsilon=4*epsilon, 
                                      verbose=verbose) 
        
        val, grad_P, grad_Q = movielensutils.total_objective(Pk, Qk, 
                                                             R, mask, rho)
        if (np.linalg.norm(grad_P) < epsilon) and (np.linalg.norm(grad_Q) < epsilon):
            break
        
    if verbose:
        print("Iter %d. Obj: %f" %(k, val))
    
    return Pk, Qk

In [35]:
R, mask = movielensutils.load_movielens('./ml-100k/u.data', 
                                        minidata=False)
C = 4
rho = 0.3
Q0, sigmas, P0 = ssl.svds(A=R, k=C)


gp = lambda P, Q: objective(P=P, 
                        Q0=Q, 
                        R=R, 
                        mask=mask, rho=rho)
gq = lambda P, Q: objective_Q(P0=P, 
                        Q=Q, 
                        R=R, 
                        mask=mask, rho=rho)  

In [36]:
%%time
Pag, Qag = alternating_gradient(objp=gp,
                                objq=gq,
                             P0=P0,
                             Q0=Q0,
                             gamma=gamma, 
                             epsilon=100, verbose=True, max_iter=60)

Iter: 1, gamma: 0.500000, ||grad|| = 735.422318
Obj: 685091.782648
Iter: 2, gamma: 0.500000, ||grad|| = 406.726326
Obj: 476377.423776
Iter: 3, gamma: 0.500000, ||grad|| = 240.622838
Obj: 410948.625792
Iter: 4, gamma: 0.500000, ||grad|| = 151.250253
Obj: 387501.076358
Iter: 5, gamma: 0.500000, ||grad|| = 99.963810
Obj: 378043.960249
Iter: 6, gamma: 0.500000, ||grad|| = 68.729358
Obj: 373842.953513
Iter: 7, gamma: 0.500000, ||grad|| = 48.718652
Obj: 371830.491180
Iter: 8, gamma: 0.500000, ||grad|| = 35.360362
Obj: 370808.672326
Iter: 9, gamma: 0.500000, ||grad|| = 26.145940
Obj: 370265.902462
Iter: 1, gamma: 0.000004, ||grad|| = 201241.046540
Obj: 369967.168039
Iter: 2, gamma: 0.000008, ||grad|| = 85615.926564
Obj: 271246.336678
Iter: 3, gamma: 0.000004, ||grad|| = 52918.340495
Obj: 239509.583083
Iter: 4, gamma: 0.000004, ||grad|| = 33297.201542
Obj: 233307.349781
Iter: 5, gamma: 0.000008, ||grad|| = 25486.539200
Obj: 230164.854642
Iter: 6, gamma: 0.000004, ||grad|| = 25532.938090
Obj: 2

Iter: 111, gamma: 0.000002, ||grad|| = 2683.503891
Obj: 216614.495551
Iter: 112, gamma: 0.000015, ||grad|| = 1106.924646
Obj: 216605.679741
Iter: 113, gamma: 0.000002, ||grad|| = 2465.175065
Obj: 216594.990517
Iter: 114, gamma: 0.000015, ||grad|| = 1077.130767
Obj: 216587.446228
Iter: 115, gamma: 0.000002, ||grad|| = 2270.271481
Obj: 216576.359583
Iter: 116, gamma: 0.000015, ||grad|| = 1049.787773
Obj: 216569.863196
Iter: 117, gamma: 0.000002, ||grad|| = 2096.573363
Obj: 216558.527813
Iter: 118, gamma: 0.000015, ||grad|| = 1024.585985
Obj: 216552.896324
Iter: 119, gamma: 0.000002, ||grad|| = 1942.075918
Obj: 216541.430245
Iter: 120, gamma: 0.000031, ||grad|| = 1001.259193
Obj: 216536.513730
Iter: 121, gamma: 0.000002, ||grad|| = 3466.001090
Obj: 216521.094414
Iter: 122, gamma: 0.000008, ||grad|| = 1075.291007
Obj: 216507.110820
Iter: 123, gamma: 0.000002, ||grad|| = 1561.820751
Obj: 216501.947406
Iter: 124, gamma: 0.000031, ||grad|| = 947.313030
Obj: 216498.561671
Iter: 125, gamma: 0.0

Iter: 230, gamma: 0.000008, ||grad|| = 571.163058
Obj: 216095.940192
Iter: 231, gamma: 0.000004, ||grad|| = 794.826307
Obj: 216094.319500
Iter: 232, gamma: 0.000004, ||grad|| = 654.742255
Obj: 216093.111773
Iter: 233, gamma: 0.000008, ||grad|| = 578.437029
Obj: 216092.031817
Iter: 234, gamma: 0.000002, ||grad|| = 835.566256
Obj: 216090.508064
Iter: 235, gamma: 0.000031, ||grad|| = 513.307181
Obj: 216089.533690
Iter: 236, gamma: 0.000002, ||grad|| = 1416.302890
Obj: 216083.889285
Iter: 237, gamma: 0.000008, ||grad|| = 537.559080
Obj: 216081.474569
Iter: 238, gamma: 0.000004, ||grad|| = 702.381339
Obj: 216079.853268
Iter: 239, gamma: 0.000004, ||grad|| = 597.358486
Obj: 216078.777694
Iter: 240, gamma: 0.000008, ||grad|| = 541.748243
Obj: 216077.788862
Iter: 241, gamma: 0.000004, ||grad|| = 732.411445
Obj: 216076.239773
Iter: 242, gamma: 0.000004, ||grad|| = 612.594866
Obj: 216075.150519
Iter: 243, gamma: 0.000008, ||grad|| = 547.965877
Obj: 216074.162598
Iter: 244, gamma: 0.000002, ||gra

Iter: 34, gamma: 0.000015, ||grad|| = 1075.903180
Obj: 158847.429403
Iter: 35, gamma: 0.000004, ||grad|| = 1700.107620
Obj: 158834.301077
Iter: 36, gamma: 0.000015, ||grad|| = 1069.045012
Obj: 158828.018039
Iter: 37, gamma: 0.000002, ||grad|| = 1992.105975
Obj: 158817.661984
Iter: 38, gamma: 0.000008, ||grad|| = 1131.305813
Obj: 158811.991450
Iter: 39, gamma: 0.000004, ||grad|| = 1356.095026
Obj: 158805.977929
Iter: 40, gamma: 0.000031, ||grad|| = 977.860607
Obj: 158801.350988
Iter: 41, gamma: 0.000002, ||grad|| = 2911.861598
Obj: 158786.712935
Iter: 42, gamma: 0.000004, ||grad|| = 1308.631427
Obj: 158775.322161
Iter: 43, gamma: 0.000015, ||grad|| = 921.307734
Obj: 158771.130556
Iter: 44, gamma: 0.000004, ||grad|| = 1495.564542
Obj: 158761.714898
Iter: 45, gamma: 0.000015, ||grad|| = 924.670228
Obj: 158756.940827
Iter: 46, gamma: 0.000002, ||grad|| = 1765.525084
Obj: 158749.484952
Iter: 47, gamma: 0.000008, ||grad|| = 988.317479
Obj: 158745.051696
Iter: 48, gamma: 0.000004, ||grad|| = 

Iter: 9, gamma: 0.000031, ||grad|| = 2427.618650
Obj: 129959.954734
Iter: 10, gamma: 0.000004, ||grad|| = 3307.533086
Obj: 129835.009247
Iter: 11, gamma: 0.000031, ||grad|| = 1718.489248
Obj: 129806.359985
Iter: 12, gamma: 0.000004, ||grad|| = 2990.915957
Obj: 129747.475000
Iter: 13, gamma: 0.000031, ||grad|| = 1451.475993
Obj: 129724.263161
Iter: 14, gamma: 0.000004, ||grad|| = 3053.556106
Obj: 129690.959264
Iter: 15, gamma: 0.000015, ||grad|| = 1325.131147
Obj: 129667.277674
Iter: 16, gamma: 0.000008, ||grad|| = 1696.366987
Obj: 129650.008998
Iter: 17, gamma: 0.000015, ||grad|| = 1296.017270
Obj: 129637.825205
Iter: 18, gamma: 0.000004, ||grad|| = 1843.693989
Obj: 129624.598580
Iter: 19, gamma: 0.000031, ||grad|| = 1076.007458
Obj: 129615.170929
Iter: 20, gamma: 0.000004, ||grad|| = 1921.334381
Obj: 129590.900638
Iter: 21, gamma: 0.000031, ||grad|| = 1006.936836
Obj: 129581.009346
Iter: 22, gamma: 0.000004, ||grad|| = 2098.717610
Obj: 129563.922683
Iter: 23, gamma: 0.000015, ||grad||

...

Iter: 23, gamma: 0.000031, ||grad|| = 457.472955
Obj: 65281.510895
Iter: 24, gamma: 0.000015, ||grad|| = 567.858131
Obj: 65277.503227
Iter: 25, gamma: 0.000061, ||grad|| = 415.250689
Obj: 65274.426374
Iter: 26, gamma: 0.000008, ||grad|| = 819.829674
Obj: 65268.414856
Iter: 27, gamma: 0.000031, ||grad|| = 432.515879
Obj: 65264.702269
Iter: 28, gamma: 0.000015, ||grad|| = 539.361423
Obj: 65261.147854
Iter: 1, gamma: 0.031250, ||grad|| = 164.997489
Obj: 65258.386376
Iter: 2, gamma: 0.062500, ||grad|| = 89.427624
Obj: 64646.660961
Iter: 3, gamma: 0.062500, ||grad|| = 51.025373
Obj: 64305.428465
Iter: 4, gamma: 0.031250, ||grad|| = 44.846346
Obj: 64191.945400
Iter: 5, gamma: 0.125000, ||grad|| = 32.423586
Obj: 64146.905300
Iter: 6, gamma: 0.031250, ||grad|| = 39.816422
Obj: 64058.100417
Iter: 7, gamma: 0.062500, ||grad|| = 22.806747
Obj: 64031.328065
Iter: 8, gamma: 0.031250, ||grad|| = 24.129750
Obj: 64007.885583
Iter: 1, gamma: 0.000015, ||grad|| = 7285.469231
Obj: 63994.788843
Iter: 2, g

Iter: 8, gamma: 0.015625, ||grad|| = 26.962335
Obj: 55724.454553
Iter: 1, gamma: 0.000031, ||grad|| = 4470.764015
Obj: 55716.739532
Iter: 2, gamma: 0.000031, ||grad|| = 1892.589046
Obj: 55364.064793
Iter: 3, gamma: 0.000031, ||grad|| = 1249.248457
Obj: 55290.655394
Iter: 4, gamma: 0.000031, ||grad|| = 959.075320
Obj: 55256.433977
Iter: 5, gamma: 0.000061, ||grad|| = 791.976248
Obj: 55235.382540
Iter: 6, gamma: 0.000015, ||grad|| = 1050.889173
Obj: 55214.226280
Iter: 7, gamma: 0.000244, ||grad|| = 529.570186
Obj: 55203.104295
Iter: 8, gamma: 0.000015, ||grad|| = 1174.651115
Obj: 55167.422735
Iter: 1, gamma: 0.031250, ||grad|| = 139.840177
Obj: 55154.678595
Iter: 2, gamma: 0.031250, ||grad|| = 66.615570
Obj: 54814.460198
Iter: 3, gamma: 0.031250, ||grad|| = 47.758966
Obj: 54731.586832
Iter: 4, gamma: 0.031250, ||grad|| = 38.751836
Obj: 54689.130408
Iter: 5, gamma: 0.031250, ||grad|| = 33.291339
Obj: 54661.798450
Iter: 6, gamma: 0.031250, ||grad|| = 29.485205
Obj: 54642.113617
Iter: 7, ga

Iter: 5, gamma: 0.000122, ||grad|| = 491.398274
Obj: 49309.766137
Iter: 6, gamma: 0.000015, ||grad|| = 821.585413
Obj: 49293.244325
Iter: 7, gamma: 0.000061, ||grad|| = 423.422315
Obj: 49285.793979
Iter: 8, gamma: 0.000031, ||grad|| = 521.195605
Obj: 49279.871734
Iter: 1, gamma: 0.015625, ||grad|| = 111.114632
Obj: 49275.053017
Iter: 2, gamma: 0.031250, ||grad|| = 55.179023
Obj: 49143.761627
Iter: 3, gamma: 0.031250, ||grad|| = 35.083440
Obj: 49081.395276
Iter: 4, gamma: 0.015625, ||grad|| = 37.124713
Obj: 49060.131319
Iter: 5, gamma: 0.031250, ||grad|| = 23.176484
Obj: 49047.854109
Iter: 6, gamma: 0.015625, ||grad|| = 24.576690
Obj: 49036.374759
Iter: 1, gamma: 0.000031, ||grad|| = 2280.148990
Obj: 49030.154872
Iter: 2, gamma: 0.000061, ||grad|| = 1179.142005
Obj: 48916.405676
Iter: 3, gamma: 0.000122, ||grad|| = 651.063522
Obj: 48859.025026
Iter: 4, gamma: 0.000031, ||grad|| = 731.820861
Obj: 48828.277173
Iter: 5, gamma: 0.000122, ||grad|| = 407.307068
Obj: 48818.462277
Iter: 6, gamm

Iter: 4, gamma: 0.007812, ||grad|| = 33.075003
Obj: 45343.459880
Iter: 1, gamma: 0.000061, ||grad|| = 1352.733192
Obj: 45337.561801
Iter: 2, gamma: 0.000061, ||grad|| = 679.563874
Obj: 45269.223186
Iter: 3, gamma: 0.000122, ||grad|| = 519.006811
Obj: 45248.550522
Iter: 4, gamma: 0.000031, ||grad|| = 685.887270
Obj: 45230.878814
Iter: 1, gamma: 0.015625, ||grad|| = 81.412018
Obj: 45221.647611
Iter: 2, gamma: 0.015625, ||grad|| = 40.494658
Obj: 45158.250694
Iter: 3, gamma: 0.031250, ||grad|| = 29.836022
Obj: 45139.514186
Iter: 4, gamma: 0.007812, ||grad|| = 36.186196
Obj: 45123.996560
Iter: 1, gamma: 0.000061, ||grad|| = 1301.111623
Obj: 45117.216803
Iter: 2, gamma: 0.000122, ||grad|| = 660.039412
Obj: 45053.076012
Iter: 3, gamma: 0.000031, ||grad|| = 756.744872
Obj: 45026.099845
Iter: 1, gamma: 0.015625, ||grad|| = 75.911273
Obj: 45014.457444
Iter: 2, gamma: 0.015625, ||grad|| = 38.434625
Obj: 44959.444101
Iter: 3, gamma: 0.031250, ||grad|| = 28.807333
Obj: 44942.674307
Iter: 4, gamma: 

Iter: 3, gamma: 0.000061, ||grad|| = 521.335781
Obj: 42527.392863
Iter: 1, gamma: 0.007812, ||grad|| = 78.228150
Obj: 42517.565800
Iter: 2, gamma: 0.015625, ||grad|| = 42.845948
Obj: 42482.660328
Iter: 3, gamma: 0.031250, ||grad|| = 25.151687
Obj: 42462.286586
Iter: 4, gamma: 0.007812, ||grad|| = 24.724158
Obj: 42449.229032
Iter: 1, gamma: 0.000061, ||grad|| = 880.111915
Obj: 42446.124656
Iter: 2, gamma: 0.000122, ||grad|| = 492.134745
Obj: 42413.181691
Iter: 1, gamma: 0.007812, ||grad|| = 61.356240
Obj: 42392.228632
Iter: 2, gamma: 0.015625, ||grad|| = 33.867110
Obj: 42371.047429
Iter: 3, gamma: 0.031250, ||grad|| = 22.037749
Obj: 42358.140338
Iter: 4, gamma: 0.003906, ||grad|| = 31.970959
Obj: 42349.837691
Iter: 1, gamma: 0.000061, ||grad|| = 860.369418
Obj: 42346.859432
Iter: 2, gamma: 0.000122, ||grad|| = 523.662335
Obj: 42315.389983
Iter: 3, gamma: 0.000061, ||grad|| = 448.394104
Obj: 42292.228832
Iter: 1, gamma: 0.007812, ||grad|| = 62.631683
Obj: 42283.960560
Iter: 2, gamma: 0.0

# Question 4.4
Comparer la méthode du gradient avec recherche linéaire et la méthode des moindres carrés alternés. 

Est-ce que les solutions sont les mêmes ? 

Est-ce que la prédiction R̂ = QP est la même ? 

## Reponse

Non, ils ne sont pas les mêmes. La prediction aussi.

In [37]:
def rel_error(P, Q, R):
    return np.linalg.norm(R - np.dot(Q, P))/np.linalg.norm(R)*100

relative_error_grad = rel_error(newP, newQ, R)
relative_error_mca = rel_error(Pag, Qag, R)

print("relative error (gradient): %f" % relative_error_grad)
print("relative error (moindres carré alterné): %f" % relative_error_mca)

relative error (gradient): 313.502992
relative error (moindres carré alterné): 277.449575


Est-ce que la valeur de l’objectif est la même ? 


In [38]:
obj_grad = movielensutils.total_objective(newP, newQ, R, mask, rho)
obj_mca = movielensutils.total_objective(Pag, Qag, R, mask, rho)

print("obj grad: %f" % obj_grad[0])
print("obj mca: %f" % obj_mca[0])

obj grad: 35972.554793
obj mca: 41929.403069


Comment se comparent les temps de calcul ?

** méthode des moindres carrés alternés prends plus de temps **

# Question 4.5
Quel film recommanderiez-vous à l’utilisateur 300 ?

In [39]:
def get_n_best_films(P, Q, mask, user, n=5):
    pred = Q[user,:].dot(P) * (1 - mask[user, :])
    films_sorted_by_rank = np.argsort(-pred)
    
    print("5 best films pour l'utilisateur %d" % user)
    for i in range(n):
        film_idx = films_sorted_by_rank[i]
        print(film_idx, pred[film_idx])
    return films_sorted_by_rank, pred

In [40]:
print("Gradient")
get_n_best_films(newP, newQ, mask, user=300, n=10);

Gradient
5 best films pour l'utilisateur 300
1448 4.73856130885
168 4.51928814863
250 4.43831140659
113 4.41941799431
1018 4.36266179115
519 4.35424459618
407 4.31744052003
1268 4.27559844431
319 4.25941784698
479 4.23748271807


In [41]:
print("Moindres carrés alternés")
get_n_best_films(Pag, Qag, mask, user=300, n=10);

Moindres carrés alternés
5 best films pour l'utilisateur 300
312 4.48140962046
168 4.40124941375
271 4.37835435922
519 4.32139805753
113 4.28990471301
1018 4.19589712544
407 4.1542738639
297 4.13503239897
429 4.11777559592
315 4.11458824101
